In [ ]:
import torch
from tensordict import TensorDict
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import  plotly.io as pio
from datetime import datetime
colors = ['rgb(0, 150, 130)', 'rgb(162, 34, 35)', 'rgb(35, 161, 224)']

#  SMARD DE/AT/LU

In [ ]:
file_path = '0_raw/Gro_handelspreise_201507010000_201807010000_Stunde.csv'
raw_df = pd.read_csv(file_path, sep=';', header=0, converters={'DE/AT/LU [€/MWh] Originalauflösungen': lambda x: float(x.replace(",", "."))})
raw_values = raw_df['DE/AT/LU [€/MWh] Originalauflösungen']
timestep_raw_values = pd.Series([x/100 for x in raw_values for _ in range(2)]).round(4)

start = datetime(2015, 7, 1, 0, 30)
end = datetime(2018, 7, 1, 0, 0)
dates = pd.date_range(start, end, freq='30min')

price_data = pd.DataFrame({'Date': dates, 'Price [$ per kW/h]': timestep_raw_values}, columns=['Date', 'Price [$ per kW/h]'])

In [ ]:
fig = px.line(data_frame=price_data, x='Date', y='Price [$ per kW/h]', title='<b>Price over all years</b>')
fig.update_layout(
    # autosize=False,
	xaxis_title="Date",
	yaxis_title="Price [$ per kW/h]",
	template="plotly_white", 
)
fig.add_vline(x=datetime(2016,7,1), line_dash="dash", line_color="red")
fig.add_vline(x=datetime(2017,7,1), line_dash="dash", line_color="red")
fig.update_xaxes(
    dtick="M3",
    tickformat="%b\n%Y")
fig.show()
# pio.write_image(fig=fig,
#                 file='price_overall.pdf',scale=1)

In [ ]:
first_year = price_data[price_data['Date']<datetime(2016,7,1,0,30)]
mean_first_year = torch.tensor(first_year['Price [$ per kW/h]'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

second_year = price_data[(price_data['Date']>=datetime(2016,7,1,0,30)) & (price_data['Date']<datetime(2017,7,1,0,30))]
mean_second_year = torch.tensor(second_year['Price [$ per kW/h]'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

third_year= price_data[(price_data['Date']>=datetime(2017,7,1,0,30)) & (price_data['Date']<datetime(2018,7,1,0,30))]
mean_third_year = torch.tensor(third_year['Price [$ per kW/h]'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

df = pd.DataFrame({
    'Time': pd.date_range(start="00:00", end="23:30", freq="30min").time,
    'First year': mean_first_year,
    'Second year': mean_second_year,
    'Third year': mean_third_year
})

df = df.melt('Time', var_name='Year', value_name='Price [$ per kW/h]')

fig = px.line(data_frame=df, x='Time', y='Price [$ per kW/h]', color="Year", color_discrete_sequence=colors, title='<b>Mean hour price over each years</b>')
fig.update_layout(xaxis=dict(dtick=6),
                  margin=dict(l=10, r=10, t=50, b=10),
                  template="plotly_white",
				  legend=dict(x=0.02, y=0.98, xanchor="left", yanchor="top"))
fig.show()
# fig.write_image('price_mean.pdf')

# Ausgrid

In [ ]:
# building_ids = [13,20,33,35,74,75,82,87,88,101,106,109,130,144,152,153,157,161,169,176]
building_ids = [88,101]

In [ ]:
file_paths = [
    '0_raw/2010-2011 Solar home electricity data.csv',
    '0_raw/2011-2012 Solar home electricity data.csv',
    '0_raw/2012-2013 Solar home electricity data.csv']

dataframe = pd.DataFrame()

for file_path in file_paths:
    df = pd.read_csv(file_path, header=1)

    if '2010-2011' in file_path:
        df.date = pd.to_datetime(df.date, format='%d-%b-%y')
    else:
        df.date = pd.to_datetime(df.date, format='%d/%m/%Y')
    
    if 'Row Quality' in df.columns:
        df.drop('Row Quality', axis=1, inplace=True)
    df.drop('Generator Capacity', axis=1, inplace=True)
    df.drop('Postcode', axis=1, inplace=True)

    # Append the modified dataframe to the list
    dataframe = pd.concat([dataframe, df], ignore_index=True)

for building_id in building_ids:
    df = dataframe[dataframe['Customer'] == building_id]
    
    df_long = df.melt(
    id_vars=["Customer", "Consumption Category", "date"],
    var_name="time",
    value_name="value"
	).replace('0:00', '24:00')
	# Step 2: Combine date and time into one datetime column
    df_long["datetime"] = pd.to_datetime(df_long["date"]) + pd.to_timedelta(df_long["time"] + ":00")

	# Step 3: Pivot so that categories become columns
    df_pivot = df_long.pivot_table(
		index=["Customer", "datetime"],
		columns="Consumption Category",
		values="value"
	).reset_index()
    
    start = datetime(2010, 7, 1, 0, 30)
    end = datetime(2013, 7, 1, 0, 0)
    dates = pd.date_range(start, end, freq='30min')

    df_full = (df_pivot.set_index("datetime")
             .reindex(dates)          # introduce missing rows
             .rename_axis("datetime")
             .reset_index()).drop(columns=['Customer'])
    df_full.index.name = None
    df_full.columns.name = None
    if 'CL' in df.columns:
        df_full['Consumption'] = (df_full['CL'] + df_full['GC']).round(3)
    else:
        df_full['Consumption'] = df_full['GC']
    df_full['Prosumption'] = (df_full['Consumption'] - df_full['GG']).round(3)

    if 'CL' in df_full.columns:
        fig = px.line(df_full, x='datetime', y=['Consumption', 'Prosumption', 'GC', 'CL', 'GG'], title=f'<b>Building {building_id}</b>')
        nan_mask = df_full[['GC', 'CL', 'GG']].isna().any(axis=1)
    else:
        fig = px.line(df_full, x='datetime', y=['Consumption', 'Prosumption', 'GC', 'GG'], title=f'<b>Building {building_id}</b>')
        nan_mask = df_full[['GC', 'GG']].isna().any(axis=1)
    fig.add_vline(x=datetime(2011,7,1), line_dash="dash", line_color="red")
    fig.add_vline(x=datetime(2012,7,1), line_dash="dash", line_color="red")
    nan_times = df_full.loc[nan_mask, "datetime"]
    if not nan_times.empty:
        fig.add_vrect(x0=nan_times.min(), x1=nan_times.max(), fillcolor="red", opacity=0.5, line_width=0)
    fig.show()
    # fig.write_image(f'{building_id}_overall.pdf')

    first_year = df_full[df_full['datetime']<datetime(2011,7,1,0,30)]
    mean_first_year = torch.tensor(first_year['GG'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

    second_year = df_full[(df_full['datetime']>=datetime(2011,7,1,0,30)) & (df_full['datetime']<datetime(2012,7,1,0,30))]
    mean_second_year = torch.tensor(second_year['GG'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

    third_year= df_full[(df_full['datetime']>=datetime(2012,7,1,0,30)) & (df_full['datetime']<datetime(2013,7,1,0,30))]
    mean_third_year = torch.tensor(third_year['GG'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

    df = pd.DataFrame({
        'Time': pd.date_range(start="00:00", end="23:30", freq="30min").time,
        'First year': mean_first_year,
        'Second year': mean_second_year,
        'Third year': mean_third_year
    })

    df = df.melt('Time', var_name='Year', value_name='GG')

    fig = px.line(data_frame=df, x='Time', y='GG', color="Year", title='<b>Mean hour PV over each year</b>')
    fig.update_layout(xaxis=dict(dtick=6),
                    margin=dict(l=10, r=10, t=50, b=10),
                    template="plotly_white",
                    legend=dict(x=0.02, y=0.98, xanchor="left", yanchor="top"))
    fig.show()
    # fig.write_image(f'{building_id}_pv_mean.pdf')

    first_year = df_full[df_full['datetime']<datetime(2011,7,1,0,30)]
    mean_first_year = torch.tensor(first_year['Consumption'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

    second_year = df_full[(df_full['datetime']>=datetime(2011,7,1,0,30)) & (df_full['datetime']<datetime(2012,7,1,0,30))]
    mean_second_year = torch.tensor(second_year['Consumption'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

    third_year= df_full[(df_full['datetime']>=datetime(2012,7,1,0,30)) & (df_full['datetime']<datetime(2013,7,1,0,30))]
    mean_third_year = torch.tensor(third_year['Consumption'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

    df = pd.DataFrame({
        'Time': pd.date_range(start="00:00", end="23:30", freq="30min").time,
        'First year': mean_first_year,
        'Second year': mean_second_year,
        'Third year': mean_third_year
    })

    df = df.melt('Time', var_name='Year', value_name='Consumption')

    fig = px.line(data_frame=df, x='Time', y='Consumption', color="Year", title='<b>Mean hour Consumption over each year</b>')
    fig.update_layout(xaxis=dict(dtick=6),
                    margin=dict(l=10, r=10, t=50, b=10),
                    template="plotly_white",
                    legend=dict(x=0.02, y=0.98, xanchor="left", yanchor="top"))
    fig.show()
    # fig.write_image(f'{building_id}_consumption_mean.pdf')